# Load Layer outputs

In [1]:
nn = 'NN'

import pickle
with open('results/' + nn + '_layer_outputs.dat', 'rb') as f:
    layer_outs,layer_outs_test= pickle.load(f)
f.close()


# Load recording and test data

In [2]:
import numpy as np
from keras.datasets import mnist
from scipy.misc import imresize

(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# Recording data
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

# downsample
factor = 1
if factor<1:
    Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtest.shape[0]):
        Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(X_pr.shape[0]):
        X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
else:
    Xtest_down = Xtest
    X_pr_down = X_pr
    
# VECTORIZE IMAGES
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255


Using Theano backend.


# Run Neural Networks and average outputs

In [6]:
from copy import copy, deepcopy
from RE_PartialRecData2 import RE_PartialRecData2
from sklearn import preprocessing, linear_model
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import EarlyStopping
from fancyimpute import SoftImpute
import time

fname = 'RE_run_NN_SI.log'

# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = [2**x for x in range(8)]
nSubnetSize = len(subnetSize)
# which layers?
#iLayer = 0
# how many samples per recording?
nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
# how many iterations
nIterations = 5

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]


for iLayer in range(2, len(layer_outs)-1,2):
    with open(fname, 'a') as flog:
        flog.write('Layer '+ str(iLayer)+'\n')
    rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])
    rmses_r = np.zeros([nIterations, nOutNeurons, nSubnetSize])
    print('Layer ', iLayer)
    for ss in range(nSubnetSize):
        #r_coefs = np.zeros((nIterations, 10, 128))
        #nLayerNeurons = [subnetSize[ss], 0, 0, 0, 10]
        nLayerNeurons = subnetSize[ss]
        with open(fname, 'a') as flog:
            flog.write(str(subnetSize[ss])+'\n')
        for it in range(nIterations):
            start = time.time()
            # copy data - is this necessary?
            layer_outputs = deepcopy(layer_outs)          
            # subsample
            X_subsample, Y_subsample = RE_PartialRecData2(layer_outputs[iLayer], layer_outputs[oLayer], 
                                                              subnetSize[ss], nRecordings,nSamples[ss])
            (keep_cols,) =np.where(~np.all(np.isnan(X_subsample), axis=0))#(~a[0, :])
#             # impute X_subsample with soft impute
            if ss==nSubnetSize-1:
                X_new = X_subsample[:,keep_cols]
            else:
                X_new = SoftImpute(convergence_threshold=0.01, max_iters=30).complete(X_subsample[:, keep_cols])
           
            X_test_new = layer_outs_test[iLayer][:, keep_cols]
            
            # run NN
            model = Sequential()
            model.add(Dense(16, input_shape=(X_new.shape[1],), activation='relu'))
            model.add(Dense(16, activation='relu'))
            model.add(Dense(16, activation='relu'))                
            model.add(Dense(10, activation='softmax'))
            # Compile model
            model.compile(loss='mean_squared_error', optimizer='adam')
            callbacks = [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0)]
            model.fit(X_new, Y_subsample, epochs=30, batch_size=16, shuffle=True, verbose=0, 
                      validation_split=0.1, callbacks=callbacks)

            rmses[it,:,ss] = np.sqrt(np.mean((model.predict(X_test_new) - layer_outs_test[oLayer])**2, axis=0))
                
            regr =linear_model.LinearRegression()
            regr.fit(X_new, Y_subsample)
            #r_coefs[it,:, ]=regr.coef_
            # calculate rmse
            rmses_r[it,:,ss] = np.sqrt(np.mean((regr.predict(X_test_new) - layer_outs_test[oLayer])**2, axis=0))
            
            stop = time.time()
            duration = stop-start
            with open(fname, 'a') as flog:
                flog.write('iteration '+str(it)+ ', ' +str(duration)+' sec'+'\n')
                
    print ('NN, predicting, mean RMSEs=%f' %np.mean(rmses[it, :, ss]))
    fName = 'results/NNSI_RMSES_Layer' + str(iLayer) + 'nRec' + str(nRecordings) + nn + '.dat'
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()
    print ('LR, predicting, mean RMSEs=%f' %np.mean(rmses_r[it, :, ss]))
    rmses =rmses_r
    fName = 'results/LRSI_RMSES_Layer' + str(iLayer) + 'nRec' + str(nRecordings) + nn + '.dat'
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()


('Layer ', 2)
[SoftImpute] Max Singular Value of X_init = 6433.180002
[SoftImpute] Iter 1: observed MAE=0.038977 rank=6
[SoftImpute] Iter 2: observed MAE=0.038977 rank=6
[SoftImpute] Iter 3: observed MAE=0.038977 rank=6
[SoftImpute] Iter 4: observed MAE=0.038977 rank=6
[SoftImpute] Iter 5: observed MAE=0.038977 rank=6
[SoftImpute] Iter 6: observed MAE=0.038977 rank=6
[SoftImpute] Iter 7: observed MAE=0.038977 rank=6
[SoftImpute] Iter 8: observed MAE=0.038977 rank=6
[SoftImpute] Iter 9: observed MAE=0.038977 rank=6
[SoftImpute] Iter 10: observed MAE=0.038977 rank=6
[SoftImpute] Iter 11: observed MAE=0.038977 rank=6
[SoftImpute] Iter 12: observed MAE=0.038977 rank=6
[SoftImpute] Iter 13: observed MAE=0.038977 rank=6
[SoftImpute] Iter 14: observed MAE=0.038977 rank=6
[SoftImpute] Iter 15: observed MAE=0.038977 rank=6
[SoftImpute] Iter 16: observed MAE=0.038977 rank=6
[SoftImpute] Iter 17: observed MAE=0.038977 rank=6
[SoftImpute] Iter 18: observed MAE=0.038977 rank=6
[SoftImpute] Iter 19:

In [4]:
'Layer '+ str(iLayer)


'Layer 2'